<a href="https://colab.research.google.com/github/anvithakp/ML-Google-Colab/blob/master/Politifact_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Used Beautiful Soup to perform webscraping on politifact.com**



In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 8.5MB/s 


In [ ]:
!pip install prettify

  Created wheel for prettify: filename=prettify-0.1.1-cp36-none-any.whl size=8424 sha256=484351bbc8ae9c742c6b08ad029ef120ac537888e6d5f9a6d188df6ba23731be
  Stored in directory: /root/.cache/pip/wheels/49/44/c3/f997cb138383a4a0399a6b2f75012746c533a24ff6b02fafe0
Successfully built prettify


In [ ]:
!pip install htmltext

In [ ]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import json

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.9.3163.100 Safari/537.36'
}

my_headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [ ]:
from urllib.parse import urlparse, parse_qs
import pandas as pd
import re
with requests.Session() as s:
  page_start = 1
  #n_pages = 600
  n_pages = 1
  seconds_pause = 2
  articles = []
  
class Article:
  def __init__(self, meter,source,content,name,ref, context, content_href):
    self.meter = meter
    self.source = source
    self.content = content
    self.name = name
    self.ref = ref
    self.context = context
    self.content_href = content_href

pm = re.compile(r"in (.*):$")
col_name = ["meter","source","statement","context","tags"]
column_names = ["true","mostly-true","half-true","barely-true","false","pants-fire"]
data_frame = pd.DataFrame(columns = col_name+column_names)

for page_number in range(page_start, n_pages + 1):
        # Get html from that page
        page_url = 'https://www.politifact.com/factchecks/list/?page={}'.format(page_number)
        print(page_url)
        r = s.get(page_url, headers=my_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        statements = soup.find_all("article", class_= 'm-statement')
        for statement in statements:
          meter = statement.attrs['class'][2].split("m-statement--",1)[1]
          source_details = statement.find("a", class_= 'm-statement__name')
          source = source_details.string.strip()
          ref = source_details['href']
          content = statement.find("div", class_ = 'm-statement__quote').find("a")
          content_href = content['href']
          content_value = content.string.strip()
          author = statement.find("footer", class_ = 'm-statement__footer').string
          author_part = author.split()
          name = author_part[1]+' ' + author_part[2]
          context_st = statement.find("div", class_ = 'm-statement__desc').string
          if pm.search(context_st) is None:
            context=""
          else:
            context = pm.search(context_st).group(1)     
          article = Article(meter,source,content_value,name, ref, context, content_href)
          articles.append(article)
          # new_row = {'meter': meter, 'source': source, 'statement':content, 'context':context}
          # data_frame = data_frame.append(new_row, ignore_index=True)
          # print(data_frame)


        # print(source,name)
        #name = speaker, content = text, meter = target, 

# print(data_frame)

column_names = ["true","mostly-true","half-true","barely-true","false","pants-fire"]
df = pd.DataFrame(columns = column_names)
data={}
for article in articles:
    page_url = "https://www.politifact.com"+ article.ref
    factcheck_url = "https://www.politifact.com"+ article.content_href

    source = urlparse(page_url).path.split('/')[2].replace('-', ' ')
    if not article.source in data:
      # Get Fact Check
      print(factcheck_url)
      r =  s.get(factcheck_url, headers=my_headers)
      soup = BeautifulSoup(r.content, 'html.parser')
      tags = soup.find_all("a", class_='c-tag')
      tag_list = []
      for tag in tags:
        tag_list.append(tag['title'])
      # 
      r = s.get(page_url, headers=my_headers)
      soup = BeautifulSoup(r.content, 'html.parser')
      scorecards = soup.find_all("p", class_='m-scorecard__checks')
      fact_scores={}
      for scorecard in scorecards:
        statement = scorecard.find("a",href=True)
        check_name=str(parse_qs(urlparse(statement['href']).query)['ruling']).replace("\'",'').replace("[","").replace("]","")
        check_value=statement.string.split()[0]
        fact_scores[str(check_name)]=check_value
      data[article.source]=fact_scores
    fact_score_list = data.get(article.source)
    new_row = {'meter': article.meter, 'source': article.source, 'statement':article.content, 'context':article.context, 'tags': tag_list,
               'true': fact_scores.get('true'), 'mostly-true': fact_scores.get('mostly-true'), 'half-true': fact_scores.get('half-true'),
               'barely-true': fact_scores.get('barely-true'), 'false': fact_scores.get('false'), 'pants-fire': fact_scores.get('pants-fire')}
    data_frame = data_frame.append(new_row, ignore_index=True)

for item in data.items():
  values = item[1]
  df.loc[item[0]]=[values['true'],values['mostly-true'],values['half-true'],values['barely-true'],values['false'],values['pants-fire']]


print(data_frame)



https://www.politifact.com/factchecks/list/?page=1
https://www.politifact.com/factchecks/2020/nov/25/instagram-posts/kyle-rittenhouse-use-covid-19-stimulus-kenosha/
https://www.politifact.com/factchecks/2020/nov/25/robin-vos/wisconsin-assembly-speaker-says-state-passed-one-f/
https://www.politifact.com/factchecks/2020/nov/24/facebook-posts/dominion-and-smartmatic-offices-havent-shuttered-a/
https://www.politifact.com/factchecks/2020/nov/24/viral-image/pennsylvania-has-banned-some-alcohol-sales-thanksg/
https://www.politifact.com/factchecks/2020/nov/24/jody-hice/did-democrats-refuse-acknowledge-trumps-2016-victo/
https://www.politifact.com/factchecks/2020/nov/23/donald-trump/no-there-have-not-been-more-votes-people-who-voted/
https://www.politifact.com/factchecks/2020/nov/23/david-perdue/ga-sen-david-perdues-claim-jon-ossoff-socialist-pa/
https://www.politifact.com/factchecks/2020/nov/23/blog-posting/no-george-soros-wasnt-arrested-philadelphia-electi/
https://www.politifact.com/factchec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data_frame.to_csv('/content/drive/My Drive/politifact_new.tsv', sep = '\t')